<a href="https://colab.research.google.com/github/Vedarham/00_MovRecom_Py/blob/main/00_Movie_Recommendation_ML_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import difflib




In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre-Processing

In [9]:
# loading data from csv file to pandas dataframe

movies_data = pd.read_csv('/content/movies.csv')

In [10]:
# printing teh first 5 rows  of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [11]:
# Number of rows and columns in the dataframe

movies_data.shape

(4803, 24)

In [90]:
# Selecting the relevant features for recommendations
selected_feature = ['genres', 'keywords', 'tagline', 'cast', 'director','overview']


In [91]:
# replacing the missing or null values with  null string

for features in selected_feature:
  movies_data[features] = movies_data[features].fillna('')


In [92]:
# Combining all the selccted features

combined_features = movies_data['genres']+ ' ' + movies_data['keywords']+ ' ' + movies_data['overview']+ ' ' + movies_data['tagline']+' ' +movies_data['cast']+' ' +movies_data['director']



In [93]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      When ambitious New York attorney Sam is sent...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [94]:
# Converting text data to feature vector
vectorizer = TfidfVectorizer()

In [95]:
feature_vectors = vectorizer.fit_transform(combined_features)


In [96]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 307355 stored elements and shape (4803, 30592)>
  Coords	Values
  (0, 561)	0.05971816344971169
  (0, 703)	0.06846420517510078
  (0, 9754)	0.08513696797398294
  (0, 23977)	0.07941905576010944
  (0, 10023)	0.07960231361105431
  (0, 6601)	0.1498786462809525
  (0, 5279)	0.1549075340655008
  (0, 10796)	0.11095111375730655
  (0, 25413)	0.24369151759694266
  (0, 29469)	0.08637114089261566
  (0, 5592)	0.17322386697661618
  (0, 25232)	0.1336739112380023
  (0, 13474)	0.03647840985795873
  (0, 27118)	0.08328687324810813
  (0, 239)	0.19716936546022962
  (0, 4768)	0.12501550204808315
  (0, 20104)	0.19260031442011954
  (0, 17021)	0.14324534514274753
  (0, 14023)	0.041700928584839175
  (0, 7827)	0.1797988202434295
  (0, 27405)	0.0310150192195571
  (0, 18249)	0.14769810641761003
  (0, 20039)	0.37747447361884223
  (0, 19541)	0.04908322232312576
  (0, 28597)	0.1549075340655008
  :	:
  (4802, 9588)	0.10415492266309316
  (4802, 21386)	0.0897551

Cosine Similarity

In [97]:
# Getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)


In [98]:
print(similarity , similarity.shape)

# Each single row is compared with all others rows and similarity score is calculated with accordance

[[1.         0.05083168 0.0332947  ... 0.02749812 0.0304889  0.0072518 ]
 [0.05083168 1.         0.04356836 ... 0.05077045 0.03100979 0.01521198]
 [0.0332947  0.04356836 1.         ... 0.02646984 0.04751623 0.01372603]
 ...
 [0.02749812 0.05077045 0.02646984 ... 1.         0.03481447 0.03546821]
 [0.0304889  0.03100979 0.04751623 ... 0.03481447 1.         0.03098945]
 [0.0072518  0.01521198 0.01372603 ... 0.03546821 0.03098945 1.        ]] (4803, 4803)


In [99]:
# Getting the movie name from the user
movie_name = input("Enter your movie name :")

Enter your movie name :looper


In [100]:
# creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [101]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Looper', 'Zookeeper', 'Closer']


In [102]:
close_match = find_close_match[0]
print(close_match)

Looper


In [103]:
# find the index of the movie with title
index_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_movie)

1568


In [104]:
# getting list of similar movies based on index values
similarity_score = list(enumerate(similarity[index_movie]))
print(similarity_score)

[(0, np.float64(0.06504957821271856)), (1, np.float64(0.08664045294032545)), (2, np.float64(0.04545142045008058)), (3, np.float64(0.05671738540348384)), (4, np.float64(0.05920218753620039)), (5, np.float64(0.030684943751289804)), (6, np.float64(0.03772000627197885)), (7, np.float64(0.06092079190532303)), (8, np.float64(0.025959648141827524)), (9, np.float64(0.03263776491198117)), (10, np.float64(0.03580776016420678)), (11, np.float64(0.040921462821254376)), (12, np.float64(0.02187746537837611)), (13, np.float64(0.04317056361990048)), (14, np.float64(0.06944006064542795)), (15, np.float64(0.04408817306548627)), (16, np.float64(0.041846619868192984)), (17, np.float64(0.05539789032493608)), (18, np.float64(0.19842038301590773)), (19, np.float64(0.05241924923182778)), (20, np.float64(0.07029204117815786)), (21, np.float64(0.03632214832687158)), (22, np.float64(0.0373428992677549)), (23, np.float64(0.030652324702589943)), (24, np.float64(0.07484290947397297)), (25, np.float64(0.040974001140

In [105]:
len(similarity_score)

4803

In [106]:
# Sorting the movies based on similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

# similarity_score is treated as "x" on basis of x[1]


[(1568, np.float64(1.0000000000000007)), (18, np.float64(0.19842038301590773)), (483, np.float64(0.182004138199725)), (46, np.float64(0.17611708590015254)), (108, np.float64(0.17328500182691964)), (4448, np.float64(0.15905920799544468)), (1399, np.float64(0.15441488774735754)), (1715, np.float64(0.15258654439886413)), (466, np.float64(0.14743710584431835)), (1096, np.float64(0.14712518912988065)), (301, np.float64(0.14662000702160033)), (2466, np.float64(0.14390455564528626)), (322, np.float64(0.14217445631535153)), (900, np.float64(0.14131414523627278)), (3552, np.float64(0.13928242853711872)), (2991, np.float64(0.13840587106382699)), (166, np.float64(0.13756065721153093)), (1398, np.float64(0.13581663321920176)), (279, np.float64(0.13309190389393036)), (1959, np.float64(0.13291223107242303)), (1535, np.float64(0.13215354064237128)), (1016, np.float64(0.1295448764748395)), (2868, np.float64(0.12859720033251165)), (311, np.float64(0.12747834767872943)), (510, np.float64(0.1262802157259

In [107]:
# Print the name of similar movies based on the index
print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<11):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Looper
2 . Men in Black 3
3 . Timeline
4 . X-Men: Days of Future Past
5 . Terminator Genisys
6 . Time Changer
7 . In Time
8 . Timecop
9 . The Time Machine
10 . Bangkok Dangerous
